In [27]:
import numpy as np
import math

# Puma Configuration
Implementation of Inverse and Forward Kinematics

In [28]:
class puma():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.a1=0.25
    self.a2=0.25
    self.a3=0.25

    self.theta1=0
    self.theta2=0
    self.theta3=0

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()

# Inverse Kinematics
  def invkine(self):
    self.theta1=np.arctan2(self.y,self.x)

    r=math.pow(math.pow(self.x,2)+math.pow(self.y,2),0.5)
    s=self.z-self.a1

    D= (math.pow(r,2)+math.pow(s,2)-math.pow(self.a2,2)-math.pow(self.a3,2)) / (2*self.a2*self.a3)
    if D>1 or D<-1 :
      print("out of the workspace")

    self.theta3 = np.arctan2((-math.pow(1-math.pow(D,2),0.5)),D)
    
    self.theta2 = np.arctan2(s,r)-np.arctan2(self.a3*np.sin(self.theta3),(self.a2+self.a3*np.cos(self.theta3)))

  def dhpara(self):
    mat = []
    mat.append([0,0,self.a1,self.theta1])
    mat.append([self.a2,np.pi/2,0,self.theta2])
    mat.append([self.a3,0,0,self.theta3])

    return mat

  def dh(self,arr):
    a=arr[0]
    alpha=arr[1]
    d=arr[2]
    theta = arr[3]

    mat = []
    b=[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)]
    mat.append(b)

    b=[np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)]
    mat.append(b)

    b=[0, np.sin(alpha), np.cos(alpha), d]
    mat.append(b)

    b=[0,0,0,1]
    mat.append(b)

    return np.matrix(mat)

# Forward Kinematics
  # def forward(self):
  #   matrix=self.dhpara()
  #   T = np.identity(4)
  #   for i in range(len(matrix)):
  #     T=np.matmul(T,self.dh(matrix[i]))
  #   return T[:3,3].T

  def forward(self):
    z1=self.a2*np.sin(self.theta2)+self.a3*np.sin(self.theta2+self.theta3)+self.a1

    r=self.a2*np.cos(self.theta2)+self.a3*np.cos(self.theta2+self.theta3)
    x1=r*np.cos(self.theta1)
    y1=r*np.sin(self.theta1)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n theta3: {self.theta3} rad \n\n")

# Stanford Configuration
Implementation of Inverse and Forward Kinematics

In [29]:
class stanford():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25

  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()
    
# Inverse Kinematics
  def invkine(self):
    if (self.x==0 and self.y==0) :
      print("Singularity !!!!!!!")
      return
    else:
      self.theta1 = np.arctan2(self.y,self.x)

    r= (self.x**2+self.y**2)**(0.5)
    s=self.z-self.a1

    self.theta2 = np.arctan2(s,r)+(np.pi/2)

    self.d3=(r**2+s**2)**(0.5) - self.a2
# Forward Kinematics
  def forward(self):
    x1=(self.a2+self.d3)*np.cos(self.theta1)
    y1=(self.a2+self.d3)*np.sin(self.theta1)
    z1=self.a1+(self.a2+self.d3)*np.sin(self.theta2)

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")


# SCARA Configuration
Implementation of Inverse and Forward Kinematics

In [30]:
class scara():
  def __init__(self):
    self.x=0
    self.y=0
    self.z=0

    self.theta1=0
    self.theta2=0
    self.d3=0

    self.a1=0.25
    self.a2=0.25
    self.h=0.25
  
  def setpos(self,arr):
    self.x=arr[0]
    self.y=arr[1]
    self.z=arr[2]
    self.invkine()
    
# Inverse Kinematics
  def invkine(self):
    r= ((self.x**2+self.y**2-self.a1**2-self.a2**2)/(2*self.a1*self.a2))
    self.theta2 = np.arctan2(((1-r**2)**0.5),r)
    self.theta1 = np.arctan2(self.y,self.x)-np.arctan2(self.a2*np.sin(self.theta2),(self.a1+self.a2*np.cos(self.theta2)))

    self.d3=self.h-self.z
# Forward Kinematics
  def forward(self):
    x1=self.a1*np.cos(self.theta1)+self.a2*np.cos(self.theta1+self.theta2)
    y1=self.a1*np.sin(self.theta1)+self.a2*np.sin(self.theta1+self.theta2)
    z1=self.h-self.d3;

    return [x1,y1,z1]

  def verify(self,arr):
    self.setpos(arr)
    print(f"Original Co-ordinates:- {[self.x,self.y,self.z]}\n")
    print(f"Verified from code :- {self.forward()}\n")
    print(f"Joint parameters: \n theta1: {self.theta1} rad \n theta2: {self.theta2} rad \n d3: {self.d3} m \n\n")

# Inputs

In [31]:
lett = ["A","B","C","D"]
c=["X","Y","Z"]
cord=[]
for i in range(len(lett)):
  print("\n")
  a=[]
  for j in range(3):
    b=float(input(f"{lett[i]} : {c[j]} "))
    a.append(b)
  cord.append(a)

cord



A : X 0.3
A : Y 0.02
A : Z 0.1


B : X 0.3
B : Y -0.02
B : Z 0.1


C : X 0.2
C : Y -0.02
C : Z 0.1


D : X 0.2
D : Y 0.02
D : Z 0.1


[[0.3, 0.02, 0.1], [0.3, -0.02, 0.1], [0.2, -0.02, 0.1], [0.2, 0.02, 0.1]]

# Verification

**Puma**

In [32]:
puma1 = puma()
for i in range(len(cord)):
  print(lett[i])
  puma1.verify(cord[i])

A
Original Co-ordinates:- [0.3, 0.02, 0.1]

Verified from code :- [0.3, 0.019999999999999997, 0.09999999999999998]

Joint parameters: 
 theta1: 0.06656816377582381 rad 
 theta2: 0.3711127907486969 rad 
 theta3: -1.6677481410165333 rad 


B
Original Co-ordinates:- [0.3, -0.02, 0.1]

Verified from code :- [0.3, -0.019999999999999997, 0.09999999999999998]

Joint parameters: 
 theta1: -0.06656816377582381 rad 
 theta2: 0.3711127907486969 rad 
 theta3: -1.6677481410165333 rad 


C
Original Co-ordinates:- [0.2, -0.02, 0.1]

Verified from code :- [0.20000000000000007, -0.020000000000000007, 0.09999999999999998]

Joint parameters: 
 theta1: -0.09966865249116202 rad 
 theta2: 0.40423729161454336 rad 
 theta3: -2.0907039852941662 rad 


D
Original Co-ordinates:- [0.2, 0.02, 0.1]

Verified from code :- [0.20000000000000007, 0.020000000000000007, 0.09999999999999998]

Joint parameters: 
 theta1: 0.09966865249116202 rad 
 theta2: 0.40423729161454336 rad 
 theta3: -2.0907039852941662 rad 




**Stanford**

In [33]:
stf=stanford()
for i in range(len(cord)):
  print(lett[i])
  stf.verify(cord[i])

A
Original Co-ordinates:- [0.3, 0.02, 0.1]

Verified from code :- [0.33526175218458204, 0.022350783478972135, 0.5506659275674581]

Joint parameters: 
 theta1: 0.06656816377582381 rad 
 theta2: 1.1080350470353268 rad 
 d3: 0.08600595232822883 m 


B
Original Co-ordinates:- [0.3, -0.02, 0.1]

Verified from code :- [0.33526175218458204, -0.022350783478972135, 0.5506659275674581]

Joint parameters: 
 theta1: -0.06656816377582381 rad 
 theta2: 1.1080350470353268 rad 
 d3: 0.08600595232822883 m 


C
Original Co-ordinates:- [0.2, -0.02, 0.1]

Verified from code :- [0.24955405771650416, -0.024955405771650414, 0.4509975124224178]

Joint parameters: 
 theta1: -0.09966865249116202 rad 
 theta2: 0.9296816257623568 rad 
 d3: 0.0007987240796890616 m 


D
Original Co-ordinates:- [0.2, 0.02, 0.1]

Verified from code :- [0.24955405771650416, 0.024955405771650414, 0.4509975124224178]

Joint parameters: 
 theta1: 0.09966865249116202 rad 
 theta2: 0.9296816257623568 rad 
 d3: 0.0007987240796890616 m 




**SCARA**

In [34]:
scr=scara()
for i in range(len(cord)):
  print(lett[i])
  scr.verify(cord[i])

A
Original Co-ordinates:- [0.3, 0.02, 0.1]

Verified from code :- [0.3, 0.01999999999999999, 0.1]

Joint parameters: 
 theta1: -0.8590611938778863 rad 
 theta2: 1.85125871530742 rad 
 d3: 0.15 m 


B
Original Co-ordinates:- [0.3, -0.02, 0.1]

Verified from code :- [0.3, -0.020000000000000046, 0.1]

Joint parameters: 
 theta1: -0.992197521429534 rad 
 theta2: 1.85125871530742 rad 
 d3: 0.15 m 


C
Original Co-ordinates:- [0.2, -0.02, 0.1]

Verified from code :- [0.20000000000000007, -0.020000000000000018, 0.1]

Joint parameters: 
 theta1: -1.2567703459800028 rad 
 theta2: 2.3142033869776815 rad 
 d3: 0.15 m 


D
Original Co-ordinates:- [0.2, 0.02, 0.1]

Verified from code :- [0.20000000000000007, 0.020000000000000018, 0.1]

Joint parameters: 
 theta1: -1.0574330409976787 rad 
 theta2: 2.3142033869776815 rad 
 d3: 0.15 m 


